In [1]:
print("Works")

Works


In [2]:
from selenium import webdriver

driver = webdriver.Chrome()


In [3]:
driver.get("https://www.rta.ae/wps/myportal/rta/ae/home/dashboard?lang=en")


In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

NOL_CARD = os.getenv("RTA_NOL_CARD")
RTA_USERNAME = os.getenv("RTA_USERNAME")
RTA_PASSWORD = os.getenv("RTA_PASSWORD")


In [5]:
from selenium.webdriver.common.by import By

username_box = driver.find_element(by=By.NAME, value="username")
password_box = driver.find_element(by=By.NAME, value="password")
login_btn = driver.find_element(by=By.ID, value="btn_login")

# username_box, password_box
username_box.send_keys(RTA_USERNAME)
password_box.send_keys(RTA_PASSWORD)
login_btn.click()

In [12]:
driver.get("https://www.rta.ae/wps/portal/rta/ae/public-transport/nol/view-history?lang=en")
enter_tag_numer_css = "#nolRefundFrm > div > div > div.swiper-wrapper > div.swiper-slide.swiper-slide-next.rtaOffice.loggedinTabs"
driver.find_element(by=By.CSS_SELECTOR, value=enter_tag_numer_css).click()
driver.find_element(by=By.NAME, value="tagId").send_keys(NOL_CARD)

input("Mark the captcha and fill it's challange, press next and let me know...")

''

In [13]:
print("Extraction...")
travel_history_css = "#smartStep_2 > div.smart-step__content > div > div > div:nth-child(5)"
travel_history =driver.find_element(by=By.CSS_SELECTOR, value=travel_history_css)

Extraction...


In [24]:
travel_history.get_attribute('innerHTML')

travel_history_table = travel_history.find_element(by=By.CLASS_NAME, value="ss-table__tbody")

In [25]:
rows_html = travel_history_table.find_elements(by=By.XPATH, value="*")
len(rows_html)

126

In [27]:
type(rows_html[0])

selenium.webdriver.remote.webelement.WebElement

In [48]:
rows_html[0].get_attribute("innerHTML")


def extract_travel(row: webdriver.remote.webelement.WebElement) -> dict:
    def extract(item: webdriver.remote.webelement.WebElement):
        return (
            item.get_attribute("innerHTML")
            .replace("\n", "")
            .replace("\t", "")
            .replace(r"\s{2,}", " ")
            .strip()
        )

    def extract_amount(item: webdriver.remote.webelement.WebElement):
        return (
            item.find_element(by=By.XPATH, value="*")
            .get_attribute("innerHTML")
            .replace("\n", "")
            .replace("\t", "")
            .replace(r"\s{2,}", " ")
            .strip()
        )

    children = row.find_elements(by=By.XPATH, value="*")
    nol_tag_id = extract(children[0])
    date = extract(children[1])
    time = extract(children[2])
    trip = extract(children[3])
    event = trip.split(":")[0].strip()
    station = trip.split(":")[1].strip()
    amount = extract_amount(children[4])
    return {
        "nol_tag_id": nol_tag_id,
        "date": date,
        "time": time,
        # "trip": trip,
        "event": event,
        "station": station,
        "amount": amount,
    }

In [49]:
travels = [extract_travel(row) for row in rows_html]
travels[:3]

[{'nol_tag_id': '0292627163',
  'date': '20/09/24',
  'time': '05:25 PM',
  'event': 'Check-out',
  'station': 'Business Bay Metro Station',
  'amount': '7.5'},
 {'nol_tag_id': '0292627163',
  'date': '20/09/24',
  'time': '05:14 PM',
  'event': 'Check-in',
  'station': 'World Trade Centre Metro Station',
  'amount': '-7.5'},
 {'nol_tag_id': '0292627163',
  'date': '20/09/24',
  'time': '08:41 AM',
  'event': 'Check-out',
  'station': 'World Trade Centre Metro Station',
  'amount': '7.5'}]

In [51]:
import pandas as pd
from datetime import datetime

date_time = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
df = pd.DataFrame(travels, columns=["nol_tag_id", "date", "time", "event", "station", "amount"])
df.to_csv(f"travel_history_{date_time}.csv", index=False)
df

,nol_tag_id,date,time,event,station,amount
0,0292627163,20/09/24,05:25 PM,Check-out,Business Bay Metro Station,7.5
1,0292627163,20/09/24,05:14 PM,Check-in,World Trade Centre Metro Station,-7.5
2,0292627163,20/09/24,08:41 AM,Check-out,World Trade Centre Metro Station,7.5
3,0292627163,20/09/24,08:31 AM,Check-in,Business Bay Metro Station,-7.5
4,0292627163,19/09/24,05:27 PM,Check-out,Business Bay Metro Station,7.5
...,...,...,...,...,...,...
121,0292627163,21/08/24,07:25 PM,Check-in,Emirates Towers Metro Station,-7.5
122,0292627163,21/08/24,06:57 PM,Check-out,Emirates Towers Metro Station,7.5
123,0292627163,21/08/24,06:52 PM,Check-in,World Trade Centre Metro Station,-7.5
124,0292627163,21/08/24,08:37 AM,Check-out,World Trade Centre Metro Station,7.5
